# Notebook error analysis 

## Libraries and exception types


### 1. nbs from github - the stack v1

https://huggingface.co/datasets/bigcode/the-stack-dedup/tree/main/data/jupyter-notebook


In [1]:
# read in all errors from the file
import pandas as pd
import util

df2_err_builtin_exps = pd.read_excel('C:/Users/yirwa29/Downloads/Dataset-Nb/nberror_g_all_p4_exception_types.xlsx')
df2_err_builtin_exps.head()

,fname,ename,evalue,traceback,lib
0,00000-101-cookie-clicker-v2-checkpoint.ipynb,keyboardinterrupt,NaN,['--------------------------------------------...,"urllib3,selenium"
1,00000-1012-demand-forecasting-data-prep-from-s...,libcustomerrors,"FATAL: no pg_hba.conf entry for host ""75.166....",['--------------------------------------------...,psycopg2
2,00000-1017-heatmapseq2seq.ipynb,valueerror,"x and y must have same first dimension, but ha...",['--------------------------------------------...,matplotlib
3,00000-1023-crawler-20190515-20190516.ipynb,connectionerror,HTTPSConnectionPool(host='www.backpackers.com....,['--------------------------------------------...,"requests,urllib3"
4,00000-1033-peer-solution-predicting-survival-t...,libcustomerrors,<urlopen error [Errno 11001] getaddrinfo failed>,['--------------------------------------------...,pandas


In [2]:
len(df2_err_builtin_exps[~df2_err_builtin_exps["lib"].isnull()])/len(df2_err_builtin_exps)

0.35296669615239423

In [3]:
df2_err_builtin_exps["lib_parsed"] = df2_err_builtin_exps.lib.map(util.simple_lib_parser)

In [4]:
df2_err_builtin_exps.lib_parsed.value_counts()[:10]

lib_parsed
pandas        13373
tensorflow     8684
torch          5842
sklearn        4398
numpy          3772
matplotlib     2611
ipython        1163
scipy           833
seaborn         680
zmq             562
Name: count, dtype: int64

In [5]:
for builtin_exp in df2_err_builtin_exps.ename.value_counts().index:
    df2_err_builtin_exp = df2_err_builtin_exps[df2_err_builtin_exps["ename"]==builtin_exp]
    libs_n = df2_err_builtin_exp.lib_parsed.value_counts()
    lib_percent = len(df2_err_builtin_exp[~df2_err_builtin_exp["lib_parsed"].isnull()])/len(df2_err_builtin_exp)
    n = min(3, len(libs_n))
    print("For {0}({1}), {2:.2%} are with libraries, the top {3} are:".format(builtin_exp, len(df2_err_builtin_exp),lib_percent, n))
    for i in range(n):
        print("\t{0:<12} {1:>12} samples".format(libs_n.index[i], libs_n.values[i]))

For nameerror(36962), 1.11% are with libraries, the top 3 are:
	ipython                71 samples
	pandas                 52 samples
	tensorflow             35 samples
For typeerror(23563), 21.57% are with libraries, the top 3 are:
	pandas               1398 samples
	numpy                 465 samples
	matplotlib            457 samples
For keyboardinterrupt(18781), 78.71% are with libraries, the top 3 are:
	tensorflow           4133 samples
	torch                2624 samples
	sklearn              1025 samples
For valueerror(18143), 64.26% are with libraries, the top 3 are:
	sklearn              2415 samples
	pandas               2368 samples
	tensorflow           1848 samples
For attributeerror(12924), 28.88% are with libraries, the top 3 are:
	pandas               1534 samples
	tensorflow            298 samples
	torch                 246 samples
For libcustomerrors(12804), 71.45% are with libraries, the top 3 are:
	tensorflow           1094 samples
	pandas                709 samples
	z

In [6]:
import util

util.print_traceback(df2_err_builtin_exps["traceback"][(df2_err_builtin_exps["ename"]=="valueerror")&df2_err_builtin_exps["lib_parsed"].isnull()].iloc[10])

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-178-9e32465e26db> in <module>()
      1 """Train the model after regularization"""
----> 2 parameters,costs = L_layer_model(X_train, y_train, layers_dims, learning_rate = 0.075, num_iterations = 1400, print_cost = True,lambd = 0,keep_prob=1,grad_check=True)

<ipython-input-157-bb1b26ff3782> in L_layer_model(X, Y, layers_dims, learning_rate, num_iterations, print_cost, lambd, keep_prob, grad_check)
     28         ### START CODE HERE ### (≈ 1 line of code)
     29         if keep_prob == 1:
---> 30             AL, caches = L_model_forward(X,parameters,keep_prob)
     31         elif keep_prob < 1:
     32             AL, caches = L_model_forward(X,parameters,keep_prob)

<ipython-input-131-03adf842c8b4> in L_model_forward(X, parameters, keep_prob)
     23 
     24         ### START CODE HERE ### (≈ 2 lines of code)
---> 25 

### 2. nbs from kaggle


### 2.1 Popular libraries used in data science and ML notebooks
Extract imported libraries from the ENTIRE Kaggle notebooks

In [5]:
import imports_parser
import pandas as pd

path_test = r"C:\Users\yirwa29\Downloads\Dataset-Nb\nbdata_k_error\230102"
res = imports_parser.get_imports_nbs_outermost_static(path_test)
pd.DataFrame.from_dict(res)

,fname,imports
0,aadyac_ingenium-level3.ipynb,"{keras, pandas, matplotlib, tensorflow, numpy,..."
1,abaojiang_eda-on-game-progress.ipynb,"{typing, pandas, matplotlib, seaborn, warnings..."
2,abdallahelsayed22_image-segmentation-u-net.ipynb,"{PIL, glob, keras, matplotlib, tensorflow, skl..."
3,abdallahwagih_plant-stress-identification-acc-...,"{pandas, matplotlib, pathlib, tensorflow, warn..."
4,abdelrahmanmuhsen_semseg-tests.ipynb,"{PIL, keras, matplotlib, seaborn, tensorflow, ..."
...,...,...
621,ziadharoon11_imdb-not-preprocessed-training.ipynb,"{copy, pandas, matplotlib, transformers, nltk,..."
622,zoecompaore_final-task-bootcamp.ipynb,"{pandas, matplotlib, seaborn, numpy, os, stats..."
623,zuojianfeng_siim-mmdetection-cascadercnn-weigh...,"{cv2, pandas, json, matplotlib, kaggle_secrets..."
624,zymzym_04-pytorch-custom-datasets.ipynb,"{PIL, torchvision, typing, pandas, matplotlib,..."
